Code taken/adapted from:
* https://www.youtube.com/watch?v=U51ranzJBpY [ TOKENISER ]
* https://www.kaggle.com/abhishek/bert-base-uncased-using-pytorch [ TRAINING ]
* https://www.kaggle.com/abhishek/roberta-inference-5-folds [ INFERENCE ] 
* https://www.kaggle.com/masterscrat/detect-if-notebook-is-running-interactively [ CHECK WHERE NOTEBOOK IS RUNNING ]
* https://www.kaggle.com/c/tweet-sentiment-extraction/discussion/141502 [ SUBMISSION ]

* same as xlnet_model_0508_1.ipynb but adds feature representing how string was encoded in csv input files

In [15]:
!pip install transformers
!pip install tokenizers
!pip install protobuf

In [16]:
try:
    from google.colab import drive
    IN_COLAB = True
    drive.mount('/content/drive')
    !wget https://raw.githubusercontent.com/google/sentencepiece/master/python/sentencepiece_pb2.py
    !wget https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-spiece.model
except:
    IN_COLAB = False
    
    import sys
    sys.path.append('/kaggle/input/sentencepiece-pb2/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
--2020-05-14 20:21:05--  https://raw.githubusercontent.com/google/sentencepiece/master/python/sentencepiece_pb2.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7382 (7.2K) [text/plain]
Saving to: ‘sentencepiece_pb2.py.1’

sentencepiece_pb2.p 100%[===================>]   7.21K  --.-KB/s    in 0s      

2020-05-14 20:21:05 (86.0 MB/s) - ‘sentencepiece_pb2.py.1’ saved [7382/7382]

--2020-05-14 20:21:07--  https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-spiece.model
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.80.11
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.80.11|:443... connected.
HTTP requ

## Import library

In [0]:
from pathlib import Path
import numpy as np
import pandas as pd
import os
import tokenizers
import string
import torch
import transformers
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import re
import sentencepiece as spm
import sentencepiece_pb2

In [18]:
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [0]:
class SentencePieceTokenizer:
    def __init__(self, model_name):
        self.sp = spm.SentencePieceProcessor()
        self.sp.load(model_name)
    
    def encode(self, sentence):
        spt = sentencepiece_pb2.SentencePieceText()
        spt.ParseFromString(self.sp.encode_as_serialized_proto(sentence))
        offsets = []
        ids = []
        for piece in spt.pieces:
            ids.append(piece.id)
            offsets.append((piece.begin, piece.end))
        return {'ids' : ids,
                'offsets' : offsets}

In [0]:
class config:
    MAX_LEN = 128
    TRAIN_BATCH_SIZE = 32 #64
    VALID_BATCH_SIZE =  16
    EPOCHS = 10
    
    MODEL_CONFIG = transformers.XLNetConfig
    MODEL = transformers.XLNetForQuestionAnswering
    if IN_COLAB:
        
        BASE_PATH = Path.cwd() / "drive" / "My Drive" / "kaggle" / "tweet_sentiment_extraction"
        PRETRAINED_MODEL_DIR = BASE_PATH / "input" / "xlnetbasecased"
        TOKENIZER = SentencePieceTokenizer(str(PRETRAINED_MODEL_DIR / 'xlnet-base-cased-spiece.model'))
        MODEL_PATH = BASE_PATH  / "model_save" / "model_0508_1"
        TRAINING_FILE = BASE_PATH / "input" / "train-5fold" / "train_folds.csv"
        RAW_TRAINING_FILE = BASE_PATH / "input" / "train.csv"
        TESTING_FILE = BASE_PATH  / "input" / "test.csv"
        SAMPLE_SUBMISSION_FILE = BASE_PATH / "input" / "sample_submission.csv"
        SUBMISSION_FILE = BASE_PATH / "input" / "submission.csv"
    else:
        BASE_PATH = Path('/kaggle')
        PRETRAINED_MODEL_DIR = BASE_PATH / "input" / "xlnetbasecased"
        TOKENIZER = SentencePieceTokenizer( str(PRETRAINED_MODEL_DIR / "xlnet-base-cased-spiece.model"))
        MODEL_PATH = BASE_PATH  / "input" / "xlnetmodel05081"
        TRAINING_FILE = BASE_PATH / "input" / "trainfolds" / "train_folds.csv"
        RAW_TRAINING_FILE = BASE_PATH / "input" / "train.csv"
        TESTING_FILE = BASE_PATH  / "input" / "tweet-sentiment-extraction" / "test.csv"
        SAMPLE_SUBMISSION_FILE = BASE_PATH / "input" / "tweet-sentiment-extraction" / "sample_submission.csv"
        SUBMISSION_FILE = BASE_PATH / "working" / "submission.csv"

In [21]:
[config.TOKENIZER.sp.id_to_piece(x) for x in range(0,30)]

['<unk>',
 '<s>',
 '</s>',
 '<cls>',
 '<sep>',
 '<pad>',
 '<mask>',
 '<eod>',
 '<eop>',
 '.',
 '(',
 ')',
 '"',
 '-',
 '–',
 '£',
 '€',
 '▁',
 '▁the',
 ',',
 '▁of',
 '▁and',
 '▁to',
 's',
 '▁a',
 '▁in',
 "'",
 '▁is',
 '▁for',
 '▁that']

In [22]:
[config.TOKENIZER.sp.piece_to_id(x) for x in ['positive', 'negative', 'neutral']]

[19036, 25976, 24734]

## Utils

In [0]:
class AverageMeter:
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def jaccard(str1, str2):
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


class EarlyStopping:
    # https://github.com/Bjarten/early-stopping-pytorch/blob/master/pytorchtools.py
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model, name):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, name)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model, name)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, name):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), name)
        self.val_loss_min = val_loss

## Data processing

In [0]:
def process_data(tweet, selected_text, sentiment, tweet_first_char, tokenizer, max_len):
    """
    Preprocessing the data to the XLNet model formatting
    """
    tweet = " ".join(str(tweet).split()) # " " + 
    selected_text =  " ".join(str(selected_text).split()) # " " +

    # find start and indices of selected_text in tweet
    len_st = len(selected_text)
    idx0 = None
    idx1 = None

    for ind in (i for i, e in enumerate(tweet) if e == selected_text[0]): # 1
        if tweet[ind: ind+len_st] == selected_text: #" " + 
            idx0 = ind
            idx1 = ind + len_st - 1
            break

    # create character mask for selected_text in tweet
    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1

    tok_tweet = tokenizer.encode(tweet)
    
    input_ids_orig = tok_tweet['ids']
    tweet_offsets = tok_tweet['offsets']
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]

    #######
    sentiment_id = {
        'positive': 19036,
        'negative': 25976,
        'neutral': 24734
    }
    first_char_id = {
        True: 12,
        False: 34
    }
    #######
    
    # https://huggingface.co/transformers/model_doc/xlnet.html#transformers.XLNetTokenizer.build_inputs_with_special_tokens
    input_ids = [sentiment_id[sentiment]] + [4] + input_ids_orig + [4] + [3]
    #input_ids = [0] + [sentiment_id[sentiment]] + [2] + [2] + input_ids_orig + [2]
    token_type_ids = [0]*2 + [1] * (len(input_ids_orig)+1) + [2]
    mask = [1] * len(token_type_ids)
    tweet_offsets = [(0, 0)] * 2 + tweet_offsets + [(0, 0)] * 2
    targets_start += 2
    targets_end += 2

    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([5] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        tweet_offsets = tweet_offsets + ([(0, 0)] * padding_length)
    
    return {
        'ids': input_ids,
        'mask': mask,
        'token_type_ids': token_type_ids,
        'targets_start': targets_start,
        'targets_end': targets_end,
        'orig_tweet': tweet,
        'orig_selected': selected_text,
        'sentiment': sentiment,
        'offsets': tweet_offsets
    }

## Data loader

In [0]:
class TweetDataset:
    def __init__(self, tweet, sentiment, selected_text, tweet_first_char):
        self.tweet = tweet
        self.sentiment = sentiment
        self.selected_text = selected_text
        self.tweet_first_char = tweet_first_char
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_LEN
    
    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, item):
        data = process_data(
            self.tweet[item], 
            self.selected_text[item], 
            self.sentiment[item],
            self.tweet_first_char[item],
            self.tokenizer,
            self.max_len
        )

        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
            'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long),
            'targets_start': torch.tensor(data["targets_start"], dtype=torch.long),
            'targets_end': torch.tensor(data["targets_end"], dtype=torch.long),
            'orig_tweet': data["orig_tweet"],
            'orig_selected': data["orig_selected"],
            'sentiment': data["sentiment"],
            'offsets': torch.tensor(data["offsets"], dtype=torch.long)
        }

## Loss function

In [0]:
# def loss_fn(start_logits, end_logits, start_positions, end_positions):
#     loss_fct = nn.CrossEntropyLoss()
#     start_loss = loss_fct(start_logits, start_positions)
#     end_loss = loss_fct(end_logits, end_positions)
#     total_loss = (start_loss + end_loss)
#     return total_loss

def loss_fn(start_logprobs, end_logprobs, start_positions, end_positions):
    loss_fct = nn.NLLLoss()
    start_loss = loss_fct(start_logits, start_positions)
    end_loss = loss_fct(end_logits, end_positions)
    total_loss = (start_loss + end_loss)
    return total_loss
    

## Training function

In [0]:
def train_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()
    losses = AverageMeter()
    jaccards = AverageMeter()

    tk0 = tqdm(data_loader, total=len(data_loader))
    
    for bi, d in enumerate(tk0):

        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        sentiment = d["sentiment"]
        orig_selected = d["orig_selected"]
        orig_tweet = d["orig_tweet"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        offsets = d["offsets"]

        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets_start = targets_start.to(device, dtype=torch.long)
        targets_end = targets_end.to(device, dtype=torch.long)

        model.zero_grad()
        
        # outputs_start, outputs_end = model(
        #     ids=ids,
        #     mask=mask,
        #     token_type_ids=token_type_ids,
        # )
        # loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
        # loss.backward()

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            token_type_ids=token_type_ids,
            start_positions=targets_start, 
            end_positions=targets_end
        )
        
        loss = outputs[0]
        loss.backward()

        optimizer.step()
        scheduler.step()

        # outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
        # outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
        
        # jaccard_scores = []
        # for px, tweet in enumerate(orig_tweet):
        #     selected_tweet = orig_selected[px]
        #     tweet_sentiment = sentiment[px]
        #     jaccard_score, _ = calculate_jaccard_score(
        #         original_tweet=tweet,
        #         target_string=selected_tweet,
        #         sentiment_val=tweet_sentiment,
        #         idx_start=np.argmax(outputs_start[px, :]),
        #         idx_end=np.argmax(outputs_end[px, :]),
        #         offsets=offsets[px]
        #     )
        #     jaccard_scores.append(jaccard_score)

        # jaccards.update(np.mean(jaccard_scores), ids.size(0))
        losses.update(loss.item(), ids.size(0))
        tk0.set_postfix(loss=losses.avg)#, jaccard=jaccards.avg)

## Evaluation function

In [0]:
def calculate_jaccard_score(
    original_tweet, 
    target_string, 
    sentiment_val, 
    idx_start, 
    idx_end, 
    offsets,
    verbose=False):
    
    if idx_end < idx_start:
        idx_end = idx_start
    
    filtered_output  = ""
    for ix in range(idx_start, idx_end + 1):
        filtered_output += original_tweet[offsets[ix][0]: offsets[ix][1]]

        # add spacing to output
        if (ix+1) < len(offsets) and offsets[ix][1] < offsets[ix+1][0]:
            filtered_output += " "

    if sentiment_val == "neutral" or len(original_tweet.split()) < 2:
        filtered_output = original_tweet

    if sentiment_val != "neutral" and verbose == True:
        if filtered_output.strip().lower() != target_string.strip().lower():
            print("********************************")
            print(f"Output= {filtered_output.strip()}")
            print(f"Target= {target_string.strip()}")
            print(f"Tweet= {original_tweet.strip()}")
            print("********************************")

    jac = jaccard(target_string.strip(), filtered_output.strip())
    return jac, filtered_output


def eval_fn(data_loader, model, device):
    model.eval()
    losses = AverageMeter()
    jaccards = AverageMeter()
    
    with torch.no_grad():
        tk0 = tqdm(data_loader, total=len(data_loader))
        for bi, d in enumerate(tk0):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            sentiment = d["sentiment"]
            orig_selected = d["orig_selected"]
            orig_tweet = d["orig_tweet"]
            targets_start = d["targets_start"]
            targets_end = d["targets_end"]
            offsets = d["offsets"]

            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets_start = targets_start.to(device, dtype=torch.long)
            targets_end = targets_end.to(device, dtype=torch.long)

            outputs = model(
                input_ids=ids,
                attention_mask=mask,
                token_type_ids=token_type_ids,
                start_positions=targets_start,
                end_positions=targets_end
            )
            loss = outputs[0]
            
            # run it again to get the probabilities
            # https://huggingface.co/transformers/model_doc/xlnet.html#xlnetforquestionanswering
            outputs = model(
                input_ids=ids,
                attention_mask=mask,
                token_type_ids=token_type_ids
            )
            # start_top_index contains the model.start_n_top highest probability starting sequence positions, in decreasing order of probability
            start_top_probs = outputs[0]  
          
            # start_top_probs contain those positions' probabilities
            # the documentation claims that the values are log probabilities, which seems to be incorrect given that the values are in [0-1]
            start_top_index = outputs[1] 

            # the i-th element of start_top_index, start_top_probs are associated with elements j*model.start_n_top+i (j=1...model.end_n_top) of end_top_index, end_top_probs, where j represents the j-th highest probability end position  
            # and NOT with the i*END_N_TOP+j elements as used here https://github.com/huggingface/transformers/blob/master/src/transformers/data/metrics/squad_metrics.py#L639
            # this can be verified by checking summation to unity
            end_top_probs = outputs[2] 
            end_top_index = outputs[3] 
            
            # calculate joint probability of start, end position tuples
            start_end_probs = (start_top_probs.repeat(1, model.end_n_top)*end_top_probs)

            # reshape so that probabilities are ordered by sequence position rather than probability so that we can combine with output of other models
            mapping_to_flat_sequence_position = (end_top_index*torch.tensor(model.start_n_top)).add(start_top_index.repeat(1, model.end_n_top))
            _, indices = torch.sort(mapping_to_flat_sequence_position, dim=1)

            start_end_probs_sorted = start_end_probs[torch.repeat_interleave(torch.arange(start_end_probs.shape[0]), start_end_probs.shape[1]).view(start_end_probs.shape),
                      indices]

            # get (flat) position in sequence of highest probability tuple
            top_start_end_probs_sorted = start_end_probs_sorted.argmax(dim=1)

            # convert flat position to separate start and end positions
            start_top_positions = (top_start_end_probs_sorted % torch.tensor(config.MAX_LEN)).cpu().detach().numpy()
            end_top_positions = (top_start_end_probs_sorted // torch.tensor(config.MAX_LEN)).cpu().detach().numpy()
            
            jaccard_scores = []
            for px, tweet in enumerate(orig_tweet):
                selected_tweet = orig_selected[px]
                tweet_sentiment = sentiment[px]
                start_top_position = start_top_positions[px]
                end_top_position = end_top_positions[px]
            
                jaccard_score, _ = calculate_jaccard_score(
                    original_tweet=tweet,
                    target_string=selected_tweet,
                    sentiment_val=tweet_sentiment,
                    idx_start=start_top_position,
                    idx_end=end_top_position,
                    offsets=offsets[px]
                )
                jaccard_scores.append(jaccard_score)

            jaccards.update(np.mean(jaccard_scores), ids.size(0))
            losses.update(loss.item(), ids.size(0))
            tk0.set_postfix(loss=losses.avg, jaccard=jaccards.avg)
    
    print(f"Jaccard = {jaccards.avg}")
    print(f"Loss = {losses.avg}")
    return jaccards.avg, losses.avg

## Training 

In [0]:
def init_model(config):
    model_config = config.MODEL_CONFIG.from_pretrained(config.PRETRAINED_MODEL_DIR )#/ "config.json")
    model_config.output_hidden_states = True
    model_config.start_n_top = config.MAX_LEN
    model_config.end_n_top = config.MAX_LEN
    #'/kaggle/input/xlnet-base-tf/xlnet-base-cased'
    model = config.MODEL.from_pretrained(config.PRETRAINED_MODEL_DIR, config=model_config)#, state_dict='/kaggle/input/xlnetmodel05081/model_3.bin')
    
    return model

In [0]:
def run_fold(fold):

    dfx = pd.read_csv(config.TRAINING_FILE)

    df_train = dfx[dfx.kfold != fold].reset_index(drop=True)
    df_valid = dfx[dfx.kfold == fold].reset_index(drop=True)

    train_dataset = TweetDataset(
        tweet=df_train.text.values,
        sentiment=df_train.sentiment.values,
        selected_text=df_train.selected_text.values,
        tweet_first_char = (df_train.text.str[0]==' ').values
    )

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.TRAIN_BATCH_SIZE,
        num_workers=4
    )

    valid_dataset = TweetDataset(
        tweet=df_valid.text.values,
        sentiment=df_valid.sentiment.values,
        selected_text=df_valid.selected_text.values,
        tweet_first_char = (df_valid.text.str[0]==' ').values
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=config.VALID_BATCH_SIZE,
        num_workers=2
    )
    
    device = torch.device("cuda")

    # initialise model
    model = init_model(config)
    
    model.to(device)

    num_train_steps = int(len(df_train) / config.TRAIN_BATCH_SIZE * config.EPOCHS)
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    ]
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=0, 
        num_training_steps=num_train_steps
    )

    es = EarlyStopping(patience=2, verbose=True)
    print(f"Training is Starting for fold={fold}")
    
    for epoch in range(config.EPOCHS):
        train_fn(train_data_loader, model, optimizer, device, scheduler=scheduler)
        jaccard, loss = eval_fn(valid_data_loader, model, device)
        print(f"Jaccard Score = {jaccard}")
        print(f"Loss score = {loss}")
        es(loss, model, name=config.MODEL_PATH / f"model_{fold}.bin")
        
        if es.early_stop:
            print("Early stopping")
            break
  
    return es.val_loss_min

## Run training

In [0]:
def run_training():
  val_loss = []
  for ifold in range(5):
      q = run_fold(ifold)
      val_loss.append(q)
  print(f'Mean val loss: {np.mean(val_loss)}')

## Predict test set

In [91]:
def predict_test():
  df_test = pd.read_csv(config.TESTING_FILE)
  df_test.loc[:, "selected_text"] = df_test.text.values

  models = []

  for mf in os.listdir(config.MODEL_PATH):
    m = init_model(config)
    
    m.load_state_dict(torch.load(config.MODEL_PATH / mf))
    print(config.MODEL_PATH / mf)
    m.eval()
    # ensure we get output probabilities for all combinations of start and end position
    m.start_n_top = config.MAX_LEN
    m.end_n_top = config.MAX_LEN
    m.to(device)

    models.append(m)

  test_dataset = TweetDataset(
          tweet=df_test.text.values,
          sentiment=df_test.sentiment.values,
          selected_text=df_test.selected_text.values
          tweet_first_char = (df_test.text.str[0]==' ').values
      )

  test_data_loader = torch.utils.data.DataLoader(
      test_dataset,
      shuffle=False,
      batch_size=config.VALID_BATCH_SIZE,
      num_workers=1
  )

  final_output = []

  with torch.no_grad():
      tk0 = tqdm(test_data_loader, total=len(test_data_loader))
      for bi, d in enumerate(tk0):
          ids = d["ids"]
          token_type_ids = d["token_type_ids"]
          mask = d["mask"]
          sentiment = d["sentiment"]
          orig_selected = d["orig_selected"]
          orig_tweet = d["orig_tweet"]
          targets_start = d["targets_start"]
          targets_end = d["targets_end"]
          offsets = d["offsets"].numpy()

          ids = ids.to(device, dtype=torch.long)
          token_type_ids = token_type_ids.to(device, dtype=torch.long)
          mask = mask.to(device, dtype=torch.long)
          targets_start = targets_start.to(device, dtype=torch.long)
          targets_end = targets_end.to(device, dtype=torch.long)
          
          summed_start_end_probs_sorted = torch.zeros(ids.shape[0], config.MAX_LEN*config.MAX_LEN).to(device)

          for model in models: 
            # run it again to get the probabilities
            # https://huggingface.co/transformers/model_doc/xlnet.html#xlnetforquestionanswering
            outputs = model(
                input_ids=ids,
                attention_mask=mask,
                token_type_ids=token_type_ids
            )

            # start_top_index contains the model.start_n_top highest probability starting sequence positions, in decreasing order of probability
            start_top_probs = outputs[0]  
          
            # start_top_probs contain those positions' probabilities
            # the documentation claims that the values are log probabilities, which seems to be incorrect given that the values are in [0-1]
            start_top_index = outputs[1] 

            # the i-th element of start_top_index, start_top_probs are associated with elements j*model.start_n_top+i (j=1...model.end_n_top) of end_top_index, end_top_probs, where j represents the j-th highest probability end position  
            # and NOT with the i*END_N_TOP+j elements as used here https://github.com/huggingface/transformers/blob/master/src/transformers/data/metrics/squad_metrics.py#L639
            # this can be verified by checking summation to unity
            end_top_probs = outputs[2] 
            end_top_index = outputs[3] 
            
            # calculate joint probability of start, end position tuples
            start_end_probs = (start_top_probs.repeat(1, model.end_n_top)*end_top_probs)

            # reshape so that probabilities are ordered by sequence position rather than probability so that we can combine with output of other models
            mapping_to_flat_sequence_position = (end_top_index*torch.tensor(model.start_n_top)).add(start_top_index.repeat(1, model.end_n_top))
            _, indices = torch.sort(mapping_to_flat_sequence_position, dim=1)

            #start_end_probs_sorted = start_end_probs[torch.arange(start_end_probs.shape[0]), indices]
            start_end_probs_sorted = start_end_probs[torch.repeat_interleave(torch.arange(start_end_probs.shape[0]), start_end_probs.shape[1]).view(start_end_probs.shape),
                      indices]

            summed_start_end_probs_sorted += start_end_probs_sorted

          avg_start_end_probs_sorted = summed_start_end_probs_sorted/torch.tensor(len(models))

          # get (flat) position in sequence of highest probability tuple
          top_avg_start_end_probs_sorted = avg_start_end_probs_sorted.argmax(dim=1)

          # convert flat position to separate start and end positions
          start_top_positions = (top_avg_start_end_probs_sorted % torch.tensor(config.MAX_LEN).to(device)).cpu().detach().numpy()
          end_top_positions = (top_avg_start_end_probs_sorted // torch.tensor(config.MAX_LEN).to(device)).cpu().detach().numpy()
          
          jaccard_scores = []
          for px, tweet in enumerate(orig_tweet):
              selected_tweet = orig_selected[px]
              tweet_sentiment = sentiment[px]
              _, output_sentence = calculate_jaccard_score(
                  original_tweet=tweet,
                  target_string=selected_tweet,
                  sentiment_val=tweet_sentiment,
                  idx_start=start_top_positions[px],
                  idx_end=end_top_positions[px],
                  offsets=offsets[px],
                  verbose=True
              )
              final_output.append(output_sentence)


  sample = pd.read_csv(config.SAMPLE_SUBMISSION_FILE)
  sample.loc[:, 'selected_text'] = final_output
  sample.to_csv("submission.csv", index=False)


SyntaxError: ignored

In [0]:
IN_KAGGLE_COMMIT = False
if (not IN_COLAB) and ('runtime' not in get_ipython().config.IPKernelApp.connection_file):
   IN_KAGGLE_COMMIT = True


print(IN_KAGGLE_COMMIT)

In [0]:
 %%time
 
if IN_COLAB:
    run_training()

if IN_KAGGLE_COMMIT:
    predict_test()

/kaggle/input/xlnetmodel05081/model_3.bin
/kaggle/input/xlnetmodel05081/model_2.bin
/kaggle/input/xlnetmodel05081/model_0.bin
/kaggle/input/xlnetmodel05081/model_1.bin


  0%|          | 0/221 [00:00<?, ?it/s]

/kaggle/input/xlnetmodel05081/model_4.bin


  0%|          | 1/221 [00:02<09:06,  2.49s/it]

********************************
Output= exciting
Target= Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China: (SH) (BJ).
Tweet= Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China: (SH) (BJ).
********************************
********************************
Output= such a shame!
Target= Recession hit Veronique Branquinho, she has to quit her company, such a shame!
Tweet= Recession hit Veronique Branquinho, she has to quit her company, such a shame!
********************************
********************************
Output= happy
Target= happy bday!
Tweet= happy bday!
********************************
********************************
Output= like
Target= http://twitpic.com/4w75p - I like it!!
Tweet= http://twitpic.com/4w75p - I like it!!
********************************
********************************
Output= that`s great!!
Target= that`s great!! weee!! visitors!
Tweet= that`s great!! weee!! visitors!
*************

...

100%|█████████▉| 220/221 [03:37<00:00,  1.02it/s]

********************************
Output= bummed
Target= so what was said?? I`m so bummed I missed it!. *sigh* I want to go to california
Tweet= so what was said?? I`m so bummed I missed it!. *sigh* I want to go to california
********************************
********************************
Output= about
Target= ï¿½ï¿½We are ready for the new generation!!...Oh yeah...We are a Backstreet fans and we are proud about it!!
Tweet= ï¿½ï¿½We are ready for the new generation!!...Oh yeah...We are a Backstreet fans and we are proud about it!!
********************************
********************************
Output= Thank you
Target= 'We never want to lose the fans that got us here!' Thank you for saying that Wish you and the others a nice day, greets
Tweet= 'We never want to lose the fans that got us here!' Thank you for saying that Wish you and the others a nice day, greets
********************************
********************************
Output= bored
Target= chillin bored drinking a margarita.

100%|██████████| 221/221 [03:38<00:00,  1.01it/s]

********************************
Output= my faves!!!
Target= munchin BACON BUTTIES! woohoo!! my faves!!!
Tweet= munchin BACON BUTTIES! woohoo!! my faves!!!
********************************
********************************
Output= chillin`
Target= no school today, teacher cancelled the lesson chillin`
Tweet= no school today, teacher cancelled the lesson chillin`
********************************
********************************
Output= hurt.
Target= My eyes are starting to hurt. So late. But must reach 200 updates due to tweets sent to youuuu! Philippines tour, pleeease?
Tweet= My eyes are starting to hurt. So late. But must reach 200 updates due to tweets sent to youuuu! Philippines tour, pleeease?
********************************
********************************
Output= best
Target= The best thing I have ever done is carry and birth a child
Tweet= The best thing I have ever done is carry and birth a child
********************************
********************************
Output= HAPPY
T

CPU times: user 2min 39s, sys: 1min 19s, total: 3min 58s
Wall time: 4min 3s


In [0]:
# test_df = pd.read_csv(config.TESTING_FILE).set_index("textID")

# sub_df = pd.read_csv    dfx = pd.read_csv(config.TRAINING_FILE)(config.SUBMISSION_FILE).set_index("textID")

# # Everything not presented in the public set 
# # will take a value of the original text
# test_df["selected_text"] = test_df.text

# # Get the public ids and assign them
# public_idxs = sub_df.index.values
# test_df.loc[public_idxs, "selected_text"] = sub_df.selected_text.values
# test_df[["selected_text"]].to_csv("submission.csv")

# Testing

In [55]:
config.TRAINING_FILE

PosixPath('/content/drive/My Drive/kaggle/tweet_sentiment_extraction/input/train-5fold/train_folds.csv')

In [0]:
dfx = pd.read_csv(config.RAW_TRAINING_FILE)


train_dataset = TweetDataset(
    tweet=dfx.text.values,
    sentiment=dfx.sentiment.values,
    selected_text=dfx.selected_text.values,
    tweet_first_char = (dfx.text.str[0]==' ').values
)

In [146]:
train_dataset.__getitem__(9)

{'ids': tensor([24734,     4,  2291,   220,    26,    46, 31618, 16427,   116,    65,
            17, 25908,  6519,    22,   616,    28, 20457,    17,   993,  1296,
           102,    63,   170,   125,   371,    36,    22,   211,    28,   325,
             4,     3,     5,     5,     5,     5,     5,     5,     5,     5,
             5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
             5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
             5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
             5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
             5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
             5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
             5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
             5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
             5,     5,     5,     5,     5,  

In [147]:
[config.TOKENIZER.sp.id_to_piece(x) for x in [24734,     4,  2291,   220,    26,    46, 31618, 16427,   116,    65,
            17, 25908,  6519,    22,   616,    28, 20457,    17,   993,  1296,
           102,    63,   170,   125,   371,    36,    22,   211,    28,   325,
             4]]

['neutral',
 '<sep>',
 '▁Why',
 '▁don',
 "'",
 't',
 '▁adobe',
 '▁realise',
 '▁no',
 '▁one',
 '▁',
 'WAN',
 'TS',
 '▁to',
 '▁pay',
 '▁for',
 '▁Photoshop',
 '▁',
 'et',
 '▁al',
 '▁so',
 '▁they',
 '▁should',
 '▁just',
 '▁give',
 '▁it',
 '▁to',
 '▁us',
 '▁for',
 '▁free',
 '<sep>']

In [0]:
dfx = pd.read_csv(config.TRAINING_FILE)#, quoting=3)#dtype={'text':object})


In [76]:
dfx.pipe(lambda x:x[x.textID=='251b6a6766']).iloc[0].text[0]#, 'text']

' '

In [85]:

(dfx.head(100).text.str[0]==' ').values

array([False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False,  True,
        True, False, False, False, False,  True, False, False, False,
        True,  True, False, False, False,  True, False,  True, False,
        True,  True, False, False,  True,  True,  True, False,  True,
       False, False,  True,  True, False, False,  True, False,  True,
       False,  True,  True,  True,  True, False,  True,  True, False,
        True,  True, False, False, False,  True,  True, False, False,
        True, False, False, False,  True, False, False, False, False,
       False,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True, False,  True,  True,  True, False,  True, False,
       False])

In [60]:
dfx.head()

,textID,text,selected_text,sentiment
0,a3d0a7d5ad,Spent the entire morning in a meeting w/ a ven...,my boss was not happy w/ them. Lots of fun.,neutral
1,251b6a6766,Oh! Good idea about putting them on ice cream,Good,positive
2,c9e8d1ef1c,says good (or should i say bad?) afternoon! h...,says good (or should i say bad?) afternoon!,neutral
3,f14f087215,i dont think you can vote anymore! i tried,i dont think you can vote anymore!,negative
4,bf7473b12d,haha better drunken tweeting you mean?,better,positive
